# Geolocalisation des arbres

In [1]:
import pandas as pd 
import numpy as np
import folium

In [2]:
df = pd.read_csv('data/trees.csv', converters={'GeoJSON': pd.read_json})

## Data Cleaning

In [16]:
df_geo = df[['GENRE_BOTA', 'ESPECE', 'VARIETE', 
             'STADEDEDEVELOPPEMENT', 'ANNEEDEPLANTATION']].copy()

gps_series = df.loc[:,'GeoJSON'].map(lambda x: tuple(x.coordinates))
df_geo['LONGITUDE'] = gps_series.str.get(0)
df_geo['LATITUDE'] = gps_series.str.get(1)

df_geo.fillna({'GENRE_BOTA' : '[Genre inconnu]',
               'ESPECE' : '[Espèce inconnue]',
               'VARIETE' : ' '}, 
              inplace=True)

df_geo.rename(columns={'GENRE_BOTA' : 'genre',
                      'STADEDEDEVELOPPEMENT' : 'stade_de_developpement',
                      'ANNEEDEPLANTATION' : 'annee_de_plantation'}, inplace=True)

df_geo.rename(columns=str.title, inplace=True)

In [17]:
df_geo.sample(n=5)

,Genre,Espece,Variete,Stade_De_Developpement,Annee_De_Plantation,Longitude,Latitude
1152,Pinus,nigra,Austriaca,Arbre adulte,1982.0,5.731615,45.173138
7343,Fraxinus,excelsior,,NaN,2017.0,5.741010,45.189735
6730,Platanus,acerifolia,,Arbre adulte,1950.0,5.717981,45.195405
29281,Acer,pseudoplatanus,Negenia,Arbre adulte,1982.0,5.736023,45.185034
29468,Alnus,cordata,,Arbre adulte,1975.0,5.738002,45.165155


## Geo Localisation des arbres

In [18]:
df_geo.loc[0,:]

Genre                       [Genre inconnu]
Espece                    [Espèce inconnue]
Variete                                    
Stade_De_Developpement                  NaN
Annee_De_Plantation                     NaN
Longitude                           5.76035
Latitude                            45.1543
Name: 0, dtype: object

In [52]:
# Carte centrée sur Grenoble
m = folium.Map(
    location=[45.17798, 5.74564], 
    zoom_start=13
)

def addToMap(x):
    folium.Marker(
        [x['Latitude'], x['Longitude']], 
        tooltip=''.join([x["Genre"], ' ', x["Espece"]]),
        icon=folium.Icon(color='green', icon='leaf')
    ).add_to(m)

df_geo.loc[df_geo['Annee_De_Plantation']==1971, :].apply(addToMap, axis=1)

m

## Heatmap de l'ensemble des arbres du dataset

In [30]:
from folium import plugins

m = folium.Map(
    location=[45.17798, 5.74564], 
    tiles='stamentoner',
    zoom_start=13
)

trees_coord = np.stack((df_geo['Latitude'], df_geo['Longitude']), axis=0).T
m.add_child(plugins.HeatMap(trees_coord, radius=8))

m

## Arbres vieillissants à Grenoble

In [47]:
m = folium.Map(
    location=[45.17798, 5.74564], 
    tiles='stamentoner',
    zoom_start=13
)


old_trees = df_geo.loc[df_geo["Stade_De_Developpement"] == 'Arbre vieillissant',
                             ['Latitude','Longitude']].values

old_trees_coord = np.stack(old_trees, axis=1).T
m.add_child(plugins.HeatMap(old_trees_coord, radius=15))

m

## Distribution des platanes 

Conclusion => le bitume fait pousser des platanes

In [54]:
m = folium.Map(
    location=[45.17798, 5.74564], 
    tiles='stamentoner',
    zoom_start=13
)

platanus_trees = df_geo.loc[df_geo["Genre"] == 'Platanus',
                             ['Latitude','Longitude']].values

platanus_trees_coord = np.stack(old_trees, axis=1).T
m.add_child(plugins.HeatMap(old_trees_coord, radius=13))

m

## Test appel API ArcGIS -> Ne pas faire trop souvent 

In [ ]:
# Ca c'est fait à l'arrache
# Et apparemment ArcGIS n'est pas trop fan... 
# Je me suis fait viré mon IP ! 

# from geopy.geocoders import ArcGIS, Nominatim
# from geopy.extra.rate_limiter import RateLimiter

# geolocator = ArcGIS(user_agent="plouf")
# data = df_geo['GeoJSON'][0:10].map(pd.read_json)
# gps_coord = data.map(lambda x: ''.join([str(x.coordinates[1]), ', ', str(x.coordinates[0])])) 
# # 1 -delay geocoding calls pour pas se faire engueuler
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.2)
# gps_coord.map(geolocator.reverse)

In [4]:
import this